# data-sources

geshun

In [ ]:
reticulate::use_condaenv("base")

## Data Sources

> Answers the question “where is the data and how do I connect to it”.

### Forms of Data

1.  Structured
2.  Semi-structured - json, html, xml
3.  Unstructured

### Data Sources

1.  Flat file
2.  Online Services - AWS, Salesforce, Snowflake
3.  In-house databases
4.  Web - Wikipedia

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3 as sq

In [2]:
con = sq.connect("data/valuation.sqlite")
cur = con.cursor()

In [4]:
sql = "select name from sqlite_master where type='table'"
cur.execute(sql)
cur.fetchall()

[('valuation_raw',), ('valuation',), ('valuation_new',)]

In [9]:
pd.read_sql("select * from valuation_new", con)

,supply_date,supplying_company,lead_valuator,product_code,expected_valuation,actual_valuation
0,2016-05-18 00:00:00,Delta Logistics,Benjamin Leeds,FOX,42770,42770
1,2018-08-04 00:00:00,Alpha Inc.,Ziion Santiago,CTC,64050,64050
2,2017-12-27 00:00:00,Gamma Ltd.,Stone Williams,WWA,107230,111070
3,2017-12-01 00:00:00,Beta Supplying Company,Benjamin Wilson,AMP,99710,99710
4,2016-03-02 00:00:00,Alpha Inc.,Rebecca Faithful,ZUK,79950,79950
...,...,...,...,...,...,...
1222,2017-11-02 00:00:00,Gamma Ltd.,Stone Williams,WWA,67310,61860
1223,2020-02-07 00:00:00,Alpha Inc.,Ziion Santiago,BNO,112540,112540
1224,2019-08-14 00:00:00,Beta Supplying Company,Richard Richardson,BNO,47210,47210
1225,2016-06-21 00:00:00,Alpha Inc.,Rebecca Faithful,WWA,57240,57240


In [10]:
pd.read_sql_table("valuation_new", con)

NotImplementedError: read_sql_table only supported for SQLAlchemy connectable.

In [ ]:
[s for s in dir(pd) if s.find("read") == 0]

['read_clipboard', 'read_csv', 'read_excel', 'read_feather', 'read_fwf', 'read_gbq', 'read_hdf', 'read_html', 'read_json', 'read_orc', 'read_parquet', 'read_pickle', 'read_sas', 'read_spss', 'read_sql', 'read_sql_query', 'read_sql_table', 'read_stata', 'read_table']

In [11]:
eng = create_engine('sqlite:///data/valuation.sqlite')
pd.read_sql_table("valuation_new", eng)

,supply_date,supplying_company,lead_valuator,product_code,expected_valuation,actual_valuation
0,2016-05-18,Delta Logistics,Benjamin Leeds,FOX,42770,42770
1,2018-08-04,Alpha Inc.,Ziion Santiago,CTC,64050,64050
2,2017-12-27,Gamma Ltd.,Stone Williams,WWA,107230,111070
3,2017-12-01,Beta Supplying Company,Benjamin Wilson,AMP,99710,99710
4,2016-03-02,Alpha Inc.,Rebecca Faithful,ZUK,79950,79950
...,...,...,...,...,...,...
1222,2017-11-02,Gamma Ltd.,Stone Williams,WWA,67310,61860
1223,2020-02-07,Alpha Inc.,Ziion Santiago,BNO,112540,112540
1224,2019-08-14,Beta Supplying Company,Richard Richardson,BNO,47210,47210
1225,2016-06-21,Alpha Inc.,Rebecca Faithful,WWA,57240,57240


In [17]:
sql = "select * from valuation_new limit 6"
pd.read_sql(sql, con)

,supply_date,supplying_company,lead_valuator,product_code,expected_valuation,actual_valuation
0,2016-05-18 00:00:00,Delta Logistics,Benjamin Leeds,FOX,42770,42770
1,2018-08-04 00:00:00,Alpha Inc.,Ziion Santiago,CTC,64050,64050
2,2017-12-27 00:00:00,Gamma Ltd.,Stone Williams,WWA,107230,111070
3,2017-12-01 00:00:00,Beta Supplying Company,Benjamin Wilson,AMP,99710,99710
4,2016-03-02 00:00:00,Alpha Inc.,Rebecca Faithful,ZUK,79950,79950
5,2019-08-22 00:00:00,Theta LLC,Richmond Apinston,FOX,85350,82310


In [ ]:
pd.read_sql(sql, eng)

              supply_date  ... actual_valuation
0     2016-05-18 00:00:00  ...            42770
1     2018-08-04 00:00:00  ...            64050
2     2017-12-27 00:00:00  ...           111070
3     2017-12-01 00:00:00  ...            99710
4     2016-03-02 00:00:00  ...            79950
...                   ...  ...              ...
1222  2017-11-02 00:00:00  ...            61860
1223  2020-02-07 00:00:00  ...           112540
1224  2019-08-14 00:00:00  ...            47210
1225  2016-06-21 00:00:00  ...            57240
1226  2019-05-23 00:00:00  ...           133670

[1227 rows x 6 columns]

In [18]:
s = """
  select 
    product_code, 
    count(product_code) as n_valuation
  from valuation_new 
  where supplying_company = 'Alpha Inc.'
  group by product_code
  order by product_code desc
"""
pd.read_sql(s, eng)

,product_code,n_valuation
0,ZUK,28
1,YMD,30
2,WWA,19
3,FOX,24
4,CTC,29
5,BNO,34
6,AMP,24
7,AAD,30


In [ ]:
valuation = pd.read_sql_table("valuation_new", eng)
valuation.product_code.value_counts()

CTC    181
AAD    165
YMD    155
AMP    151
WWA    148
FOX    146
ZUK    143
BNO    138
Name: product_code, dtype: int64

product_code
AAD    165
AMP    151
BNO    138
CTC    181
FOX    146
WWA    148
YMD    155
ZUK    143
dtype: int64

In [ ]:
cur.close()
con.close()